<a href="https://colab.research.google.com/github/stixmal/praktikum_project_ds/blob/main/%D0%9E%D0%BF%D1%82%D0%B8%D0%BC%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F%20%D0%BF%D1%80%D0%BE%D0%B8%D0%B7%D0%B2%D0%BE%D0%B4%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D1%8B%D1%85%20%D1%80%D0%B0%D1%81%D1%85%D0%BE%D0%B4%D0%BE%D0%B2%20(%D0%B2%D1%8B%D0%BF%D1%83%D1%81%D0%BA%D0%BD%D0%BE%D0%B9%20%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82)/optimization_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оптимизация производственных расходов

Чтобы оптимизировать производственные расходы, металлургический комбинат ООО «Так закаляем сталь» решил уменьшить потребление электроэнергии на этапе обработки стали. Нам предстоит построить модель, которая предскажет температуру стали.  
   
    
### Описание этапа обработки   
    
    
Сталь обрабатывают в металлическом ковше вместимостью около 100 тонн. Чтобы ковш выдерживал высокие температуры, изнутри его облицовывают огнеупорным кирпичом. Расплавленную сталь заливают в ковш и подогревают до нужной температуры графитовыми электродами. Они установлены в крышке ковша.  
    
Из сплава выводится сера (десульфурация), добавлением примесей корректируется химический состав и отбираются пробы. Сталь легируют — изменяют её состав — подавая куски сплава из бункера для сыпучих материалов или проволоку через специальный трайб-аппарат (англ. tribe, «масса»). 
    
Перед тем как первый раз ввести легирующие добавки, измеряют температуру стали и производят её химический анализ. Потом температуру на несколько минут повышают, добавляют легирующие материалы и продувают сплав инертным газом. Затем его перемешивают и снова проводят измерения. Такой цикл повторяется до достижения целевого химического состава и оптимальной температуры плавки.  
    
Тогда расплавленная сталь отправляется на доводку металла или поступает в машину непрерывной разливки. Оттуда готовый продукт выходит в виде заготовок-слябов (англ. slab, «плита»).  

## 1. Исследовательский анализ данных

In [ ]:
# импорт библиотеки pandas и numpy
import pandas as pd 
import numpy as np  

# импорт модуля display
from IPython.display import display  

# импорт библиотеки mathplot и seaborn
import matplotlib.pyplot as plt
import seaborn as sns 

# отключение предупреждений
import warnings
warnings.filterwarnings('ignore') 

# импорт mae
from sklearn.metrics import mean_absolute_error, make_scorer

# импорт скеллера
from sklearn.preprocessing import StandardScaler

# импорт модели линейной регрессии
from sklearn.linear_model import LinearRegression

# импорт модели случайного леса
from sklearn.ensemble import RandomForestRegressor

# импорт модели бустинга
from catboost import CatBoostRegressor

# импорт сетки гиперпараметров 
from sklearn.model_selection import train_test_split, GridSearchCV

***

### `data_arc`

In [ ]:
# чтение файла с данными
data_arc = pd.read_csv('/datasets/final_steel/data_arc.csv')

# визуальный осмотр  
data_arc.head(10)

,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805
3,1,2019-05-03 11:18:14,2019-05-03 11:24:19,1.659363,1.062669
4,1,2019-05-03 11:26:09,2019-05-03 11:28:37,0.692755,0.414397
5,2,2019-05-03 11:34:14,2019-05-03 11:36:31,0.438063,0.283043
6,2,2019-05-03 11:38:50,2019-05-03 11:44:28,1.296415,0.892914
7,2,2019-05-03 11:46:19,2019-05-03 11:48:25,0.490377,0.305281
8,2,2019-05-03 11:49:48,2019-05-03 11:53:18,0.827743,0.516874
9,3,2019-05-03 12:06:54,2019-05-03 12:11:34,1.062053,0.671494


In [ ]:
# инфо о таблице
data_arc.info(), 
data_arc.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14876 entries, 0 to 14875
Data columns (total 5 columns):
key                     14876 non-null int64
Начало нагрева дугой    14876 non-null object
Конец нагрева дугой     14876 non-null object
Активная мощность       14876 non-null float64
Реактивная мощность     14876 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 581.2+ KB


,key,Активная мощность,Реактивная мощность
count,14876.000000,14876.000000,14876.000000
mean,1615.220422,0.670441,0.452592
std,934.571502,0.408159,5.878702
min,1.000000,0.030002,-715.504924
25%,806.000000,0.395297,0.290991
50%,1617.000000,0.555517,0.415962
75%,2429.000000,0.857034,0.637371
max,3241.000000,3.731596,2.676388


Нагревание стали в ковше происходит иттерационно, так как номер партии `key` не меняется в течение нескольких циклов. Пропусков в данных мы не наблюдаем. Однако в реактивной мощности есть выброс - избавимся от него.

In [ ]:
data_arc = data_arc.drop(data_arc[data_arc['Реактивная мощность'] < 0].index)

Очевидно, что для предсказания конечной темпереатуры в ковше, нужно найти просуммированную затраченную энергию на нагрев в пределах иттерационного процесса. Для этого найдем разность времён и сумму мощностей. 

In [ ]:
# смена названий столбцов
data_arc.columns = ['key', 'begin_time', 'end_time', 'p_power', 'q_power']

# перевод и вычисление общего времени
data_arc['total_time'] = (pd.to_datetime(data_arc['end_time']) - pd.to_datetime(data_arc['begin_time'])) \
                .dt.total_seconds().astype(int)
data_arc = data_arc.drop(['begin_time', 'end_time'], axis=1)

In [ ]:
# группировка данных
data_arc = data_arc.groupby(['key'])['p_power', 'q_power', 'total_time'].sum()

data_arc

,p_power,q_power,total_time
key,,,
1,4.878147,3.183241,1098
2,3.052598,1.998112,811
3,2.525882,1.599076,655
4,3.209250,2.060298,741
5,3.347173,2.252643,869
...,...,...,...
3237,3.932467,2.828459,909
3238,2.106529,1.681182,546
3239,4.666754,3.111617,1216


***

### `data_gas`

In [ ]:
# чтение файла с данными
data_gas = pd.read_csv('/datasets/final_steel/data_gas.csv')

# визуальный осмотр  
data_gas.head(10)

,key,Газ 1
0,1,29.749986
1,2,12.555561
2,3,28.554793
3,4,18.841219
4,5,5.413692
5,6,11.273392
6,7,9.836267
7,8,15.171553
8,9,8.216623
9,10,6.529891


In [ ]:
# смена названий столбцов
data_gas.columns = ['key', 'gas']

In [ ]:
# инфо о таблице
data_gas.info(), 
data_gas.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3239 entries, 0 to 3238
Data columns (total 2 columns):
key    3239 non-null int64
gas    3239 non-null float64
dtypes: float64(1), int64(1)
memory usage: 50.7 KB


,key,gas
count,3239.000000,3239.000000
mean,1621.861377,11.002062
std,935.386334,6.220327
min,1.000000,0.008399
25%,812.500000,7.043089
50%,1622.000000,9.836267
75%,2431.500000,13.769915
max,3241.000000,77.995040


Пропуски отсутствуют, каких-либо экстремумов мы также не наблюдаем.

***

### `data_temp`

In [ ]:
# чтение файла с данными
data_temp = pd.read_csv('/datasets/final_steel/data_temp.csv')

# визуальный осмотр  
data_temp.head(10)

,key,Время замера,Температура
0,1,2019-05-03 11:16:18,1571.0
1,1,2019-05-03 11:25:53,1604.0
2,1,2019-05-03 11:29:11,1618.0
3,1,2019-05-03 11:30:01,1601.0
4,1,2019-05-03 11:30:39,1613.0
5,2,2019-05-03 11:37:27,1581.0
6,2,2019-05-03 11:38:00,1577.0
7,2,2019-05-03 11:49:38,1589.0
8,2,2019-05-03 11:55:50,1604.0
9,2,2019-05-03 11:58:24,1608.0


In [ ]:
# смена названий столбцов
data_temp.columns = ['key', 'time', 'temperature']

In [ ]:
# инфо о таблице
data_temp.info(), 
data_temp.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15907 entries, 0 to 15906
Data columns (total 3 columns):
key            15907 non-null int64
time           15907 non-null object
temperature    13006 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 372.9+ KB


,key,temperature
count,15907.000000,13006.000000
mean,1607.880870,1591.840920
std,942.212073,21.375851
min,1.000000,1191.000000
25%,790.000000,1581.000000
50%,1618.000000,1591.000000
75%,2427.000000,1601.000000
max,3241.000000,1705.000000


Выбросов в температуре мы не наблюдаем - всё похоже на правду. Есть пропуски внутри каких-то иттераций измерений температур. Посмотрим потом, что можно с ними сделать. Так как в роли таргета у нас выступает последняя измеренная температура - вычленим её. Заодно в качестве одного из признака выделим самую первую температуру.

In [ ]:
data_temp['time'] = pd.to_datetime(data_temp['time'])

In [ ]:
# выделение признака с первой температурой на каждой иттерации
first_temp = data_temp.pivot_table(index=['key'], values=['temperature'], aggfunc='first')
first_temp.columns = ['first_temp']
first_temp

,first_temp
key,
1,1571.0
2,1581.0
3,1596.0
4,1601.0
5,1576.0
...,...
3237,1569.0
3238,1584.0
3239,1598.0


In [ ]:
# выделение таргета
target_temp = data_temp.pivot_table(index=['key'], values=['temperature'], aggfunc='last')
target_temp.columns = ['target_temp']
target_temp

,target_temp
key,
1,1613.0
2,1602.0
3,1599.0
4,1625.0
5,1602.0
...,...
3237,1569.0
3238,1584.0
3239,1598.0


***

### `data_bulk` 

In [ ]:
# чтение файла с данными
data_bulk = pd.read_csv('/datasets/final_steel/data_bulk.csv')

# визуальный осмотр  
data_bulk.head(10)

,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
0,1,NaN,NaN,NaN,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206.0,NaN,150.0,154.0
1,2,NaN,NaN,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206.0,NaN,149.0,154.0
2,3,NaN,NaN,NaN,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.0,NaN,152.0,153.0
3,4,NaN,NaN,NaN,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207.0,NaN,153.0,154.0
4,5,NaN,NaN,NaN,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203.0,NaN,151.0,152.0
5,6,NaN,NaN,NaN,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,204.0,NaN,201.0,154.0
6,7,NaN,NaN,NaN,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,204.0,NaN,152.0,154.0
7,8,NaN,NaN,NaN,99.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,410.0,NaN,252.0,153.0
8,9,NaN,NaN,NaN,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,NaN,99.0,203.0
9,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203.0,NaN,102.0,204.0


In [ ]:
# инфо о таблице
data_bulk.info(), 
data_bulk.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 16 columns):
key        3129 non-null int64
Bulk 1     252 non-null float64
Bulk 2     22 non-null float64
Bulk 3     1298 non-null float64
Bulk 4     1014 non-null float64
Bulk 5     77 non-null float64
Bulk 6     576 non-null float64
Bulk 7     25 non-null float64
Bulk 8     1 non-null float64
Bulk 9     19 non-null float64
Bulk 10    176 non-null float64
Bulk 11    177 non-null float64
Bulk 12    2450 non-null float64
Bulk 13    18 non-null float64
Bulk 14    2806 non-null float64
Bulk 15    2248 non-null float64
dtypes: float64(15), int64(1)
memory usage: 391.2 KB


,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
count,3129.000000,252.000000,22.000000,1298.000000,1014.000000,77.000000,576.000000,25.000000,1.0,19.000000,176.000000,177.000000,2450.000000,18.000000,2806.000000,2248.000000
mean,1624.383509,39.242063,253.045455,113.879045,104.394477,107.025974,118.925347,305.600000,49.0,76.315789,83.284091,76.819209,260.471020,181.111111,170.284747,160.513345
std,933.337642,18.277654,21.180578,75.483494,48.184126,81.790646,72.057776,191.022904,NaN,21.720581,26.060347,59.655365,120.649269,46.088009,65.868652,51.765319
min,1.000000,10.000000,228.000000,6.000000,12.000000,11.000000,17.000000,47.000000,49.0,63.000000,24.000000,8.000000,53.000000,151.000000,16.000000,1.000000
25%,816.000000,27.000000,242.000000,58.000000,72.000000,70.000000,69.750000,155.000000,49.0,66.000000,64.000000,25.000000,204.000000,153.250000,119.000000,105.000000
50%,1622.000000,31.000000,251.500000,97.500000,102.000000,86.000000,100.000000,298.000000,49.0,68.000000,86.500000,64.000000,208.000000,155.500000,151.000000,160.000000
75%,2431.000000,46.000000,257.750000,152.000000,133.000000,132.000000,157.000000,406.000000,49.0,70.500000,102.000000,106.000000,316.000000,203.500000,205.750000,205.000000
max,3241.000000,185.000000,325.000000,454.000000,281.000000,603.000000,503.000000,772.000000,49.0,147.000000,159.000000,313.000000,1849.000000,305.000000,636.000000,405.000000


Из описания данных мы видим, что данные о подаче сыпучих материалов в качестве присадок в стали имеют техническое обоснование. Каждая иттерация обладает своим набором материалов в качестве добавок в свойственном ей количестве. Поэтому данные о пропусках здесь закономерны и потому подлежат заполнению только нулем, чтобы не нарушать тех.процесс и не путать нашу обучаемую модель.

In [ ]:
# заполнение нулями пропусков
data_bulk = data_bulk.fillna(value = 0)
data_bulk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 16 columns):
key        3129 non-null int64
Bulk 1     3129 non-null float64
Bulk 2     3129 non-null float64
Bulk 3     3129 non-null float64
Bulk 4     3129 non-null float64
Bulk 5     3129 non-null float64
Bulk 6     3129 non-null float64
Bulk 7     3129 non-null float64
Bulk 8     3129 non-null float64
Bulk 9     3129 non-null float64
Bulk 10    3129 non-null float64
Bulk 11    3129 non-null float64
Bulk 12    3129 non-null float64
Bulk 13    3129 non-null float64
Bulk 14    3129 non-null float64
Bulk 15    3129 non-null float64
dtypes: float64(15), int64(1)
memory usage: 391.2 KB


***

### `data_bulk_time`

In [ ]:
# чтение файла с данными
data_bulk_time = pd.read_csv('/datasets/final_steel/data_bulk_time.csv')

# визуальный осмотр  
data_bulk_time.head(10)

,key,Bulk 1,Bulk 2,Bulk 3,Bulk 4,Bulk 5,Bulk 6,Bulk 7,Bulk 8,Bulk 9,Bulk 10,Bulk 11,Bulk 12,Bulk 13,Bulk 14,Bulk 15
0,1,NaN,NaN,NaN,2019-05-03 11:21:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 11:03:52,NaN,2019-05-03 11:03:52,2019-05-03 11:03:52
1,2,NaN,NaN,NaN,2019-05-03 11:46:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 11:40:20,NaN,2019-05-03 11:40:20,2019-05-03 11:40:20
2,3,NaN,NaN,NaN,2019-05-03 12:31:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 12:09:40,NaN,2019-05-03 12:09:40,2019-05-03 12:09:40
3,4,NaN,NaN,NaN,2019-05-03 12:48:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 12:41:24,NaN,2019-05-03 12:41:24,2019-05-03 12:41:24
4,5,NaN,NaN,NaN,2019-05-03 13:18:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 13:12:56,NaN,2019-05-03 13:12:56,2019-05-03 13:12:56
5,6,NaN,NaN,NaN,2019-05-03 13:59:24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 13:53:27,NaN,2019-05-03 13:53:27,2019-05-03 13:53:27
6,7,NaN,NaN,NaN,2019-05-03 14:29:14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 14:22:19,NaN,2019-05-03 14:22:19,2019-05-03 14:22:19
7,8,NaN,NaN,NaN,2019-05-03 15:04:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 14:55:46,NaN,2019-05-03 14:55:46,2019-05-03 14:55:46
8,9,NaN,NaN,NaN,2019-05-03 15:47:34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 15:41:00,NaN,2019-05-03 15:41:00,2019-05-03 15:41:00
9,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-03 16:18:52,NaN,2019-05-03 16:18:52,2019-05-03 16:18:52


In [ ]:
# инфо о таблице
data_bulk_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 16 columns):
key        3129 non-null int64
Bulk 1     252 non-null object
Bulk 2     22 non-null object
Bulk 3     1298 non-null object
Bulk 4     1014 non-null object
Bulk 5     77 non-null object
Bulk 6     576 non-null object
Bulk 7     25 non-null object
Bulk 8     1 non-null object
Bulk 9     19 non-null object
Bulk 10    176 non-null object
Bulk 11    177 non-null object
Bulk 12    2450 non-null object
Bulk 13    18 non-null object
Bulk 14    2806 non-null object
Bulk 15    2248 non-null object
dtypes: int64(1), object(15)
memory usage: 391.2+ KB


Данная таблица аналогична таблице с объёмами сыпучих присадок с той лишь разницей, что она показывает временную констатацию факта без её продолжительности. Данная информация никак не пригодна для обучения модели, поэтому просто оставим её как есть.

***

### `data_wire`

In [ ]:
# чтение файла с данными
data_wire = pd.read_csv('/datasets/final_steel/data_wire.csv')

# визуальный осмотр  
data_wire.head(10)

,key,Wire 1,Wire 2,Wire 3,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9
0,1,60.059998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,96.052315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,91.160157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,89.063515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,89.238236,9.114560,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,99.109917,9.114560,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,98.061600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,155.282394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,113.393279,9.143681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,81.244796,12.376000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# инфо о таблице
data_wire.info(), 
data_wire.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3081 entries, 0 to 3080
Data columns (total 10 columns):
key       3081 non-null int64
Wire 1    3055 non-null float64
Wire 2    1079 non-null float64
Wire 3    63 non-null float64
Wire 4    14 non-null float64
Wire 5    1 non-null float64
Wire 6    73 non-null float64
Wire 7    11 non-null float64
Wire 8    19 non-null float64
Wire 9    29 non-null float64
dtypes: float64(9), int64(1)
memory usage: 240.8 KB


,key,Wire 1,Wire 2,Wire 3,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9
count,3081.000000,3055.000000,1079.000000,63.000000,14.000000,1.000,73.000000,11.000000,19.000000,29.000000
mean,1623.426485,100.895853,50.577323,189.482681,57.442841,15.132,48.016974,10.039007,53.625193,34.155752
std,932.996726,42.012518,39.320216,99.513444,28.824667,NaN,33.919845,8.610584,16.881728,19.931616
min,1.000000,1.918800,0.030160,0.144144,24.148801,15.132,0.034320,0.234208,45.076721,4.622800
25%,823.000000,72.115684,20.193680,95.135044,40.807002,15.132,25.053600,6.762756,46.094879,22.058401
50%,1619.000000,100.158234,40.142956,235.194977,45.234282,15.132,42.076324,9.017009,46.279999,30.066399
75%,2434.000000,126.060484,70.227558,276.252014,76.124619,15.132,64.212723,11.886057,48.089603,43.862003
max,3241.000000,330.314424,282.780152,385.008668,113.231044,15.132,180.454575,32.847674,102.762401,90.053604


Данные о проволочных материалах также имеют техническое обоснование. Заменили пропуски нулями для более объективного обучения модели.

In [ ]:
# заполнение нулями пропусков
data_wire = data_wire.fillna(value = 0)
data_wire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3081 entries, 0 to 3080
Data columns (total 10 columns):
key       3081 non-null int64
Wire 1    3081 non-null float64
Wire 2    3081 non-null float64
Wire 3    3081 non-null float64
Wire 4    3081 non-null float64
Wire 5    3081 non-null float64
Wire 6    3081 non-null float64
Wire 7    3081 non-null float64
Wire 8    3081 non-null float64
Wire 9    3081 non-null float64
dtypes: float64(9), int64(1)
memory usage: 240.8 KB


***

### `data_wire_time`

In [ ]:
# чтение файла с данными
data_wire_time = pd.read_csv('/datasets/final_steel/data_wire_time.csv')

# визуальный осмотр  
data_wire_time.head(10)

,key,Wire 1,Wire 2,Wire 3,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9
0,1,2019-05-03 11:11:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2019-05-03 11:46:10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2019-05-03 12:13:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2019-05-03 12:48:05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2019-05-03 13:18:15,2019-05-03 13:32:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,2019-05-03 13:59:02,2019-05-03 13:59:54,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,2019-05-03 14:28:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,2019-05-03 15:03:42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,2019-05-03 15:42:43,2019-05-03 15:58:12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,2019-05-03 16:20:25,2019-05-03 16:21:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# инфо о таблице
data_wire_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3081 entries, 0 to 3080
Data columns (total 10 columns):
key       3081 non-null int64
Wire 1    3055 non-null object
Wire 2    1079 non-null object
Wire 3    63 non-null object
Wire 4    14 non-null object
Wire 5    1 non-null object
Wire 6    73 non-null object
Wire 7    11 non-null object
Wire 8    19 non-null object
Wire 9    29 non-null object
dtypes: int64(1), object(9)
memory usage: 240.8+ KB


Данная констация времени подачи проволочных материалов не имеет особой ценности, поэтому также оставим данную таблицу.

***

## План решения задачи

* Провести исследовательский анализ данных и понять возможно ли решение методами машинного обучения.  
* Провести предобработку данных.  
* Обучить несколько моделей и выбрать итоговую с максимальной метрикой качества.  
* Провести степень достижения соотвествия полученных результатов с поставленными задачами.  
* Составить отчёт о результатах работы, возможных затруднений и их решений, а также нахождения нетривиальных путей решения задачи.

***

## 2. Предобработка данных

Свели предварительно все признаки и таргет в одну таблицу для правильного соотнесения всех иттераций с ключом `key`.

In [ ]:
# создание общей таблицы
df = data_arc.merge(data_gas, how='left', on='key')

In [ ]:
# присоединение таблицы first_temp и target_temp
df = df.merge(first_temp, how='left', on='key')
df = df.merge(target_temp, how='left', on='key')

In [ ]:
# присоединение таблицы data_bulk и data_wire
df = df.merge(data_bulk, how='left', on='key')
df = df.merge(data_wire, how='left', on='key')

In [ ]:
# инфо об общей таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3214 entries, 0 to 3213
Data columns (total 31 columns):
key            3214 non-null int64
p_power        3214 non-null float64
q_power        3214 non-null float64
total_time     3214 non-null int64
gas            3212 non-null float64
first_temp     3214 non-null float64
target_temp    3214 non-null float64
Bulk 1         3128 non-null float64
Bulk 2         3128 non-null float64
Bulk 3         3128 non-null float64
Bulk 4         3128 non-null float64
Bulk 5         3128 non-null float64
Bulk 6         3128 non-null float64
Bulk 7         3128 non-null float64
Bulk 8         3128 non-null float64
Bulk 9         3128 non-null float64
Bulk 10        3128 non-null float64
Bulk 11        3128 non-null float64
Bulk 12        3128 non-null float64
Bulk 13        3128 non-null float64
Bulk 14        3128 non-null float64
Bulk 15        3128 non-null float64
Wire 1         3080 non-null float64
Wire 2         3080 non-null float64
Wire 3   

Удалили образовавшиеся пропуски и лишний для обучения столбец `key`.

In [ ]:
# удаление пропусков
df = df.dropna()

# удаление столбца с ключом
df = df.drop(['key'], axis=1).reset_index(drop = True)

Во избежании утечки признака в целевой признак удалили часть одинаковых значений в столбцах `first_temp` и `target_temp`.

In [ ]:
# удаление одинаковых значений
df = df.drop(df[df['first_temp'] == df['target_temp']].index).reset_index(drop=True)

In [ ]:
# вид полученной таблицы
df

,p_power,q_power,total_time,gas,first_temp,target_temp,Bulk 1,Bulk 2,Bulk 3,Bulk 4,...,Bulk 15,Wire 1,Wire 2,Wire 3,Wire 4,Wire 5,Wire 6,Wire 7,Wire 8,Wire 9
0,4.878147,3.183241,1098,29.749986,1571.0,1613.0,0.0,0.0,0.0,43.0,...,154.0,60.059998,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.052598,1.998112,811,12.555561,1581.0,1602.0,0.0,0.0,0.0,73.0,...,154.0,96.052315,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.525882,1.599076,655,28.554793,1596.0,1599.0,0.0,0.0,0.0,34.0,...,153.0,91.160157,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.209250,2.060298,741,18.841219,1601.0,1625.0,0.0,0.0,0.0,81.0,...,154.0,89.063515,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.347173,2.252643,869,5.413692,1576.0,1602.0,0.0,0.0,0.0,78.0,...,152.0,89.238236,9.11456,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2288,3.210690,2.360777,723,7.125735,1570.0,1591.0,0.0,0.0,21.0,0.0,...,223.0,89.150879,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2289,4.203064,2.810185,940,9.412616,1554.0,1591.0,0.0,0.0,0.0,63.0,...,226.0,114.179527,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2290,2.212379,1.851269,569,6.271699,1571.0,1589.0,0.0,0.0,0.0,85.0,...,226.0,94.086723,9.04800,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2291,3.408725,2.355428,750,14.953657,1591.0,1594.0,0.0,0.0,90.0,0.0,...,207.0,118.110717,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


***

## 3. Обучение моделей

В качестве целевого признака выбрали последнюю измеренную температуру.

In [ ]:
# переменные с признаками и ответами
X = df.drop(['target_temp'], axis=1)
y = df['target_temp']

In [ ]:
# раздел на 75 % обучающей и 25 % тестовой выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=12345)

Проконтролировали размер выборок.

In [ ]:
X_train.shape, y_train.shape

((1719, 29), (1719,))

In [ ]:
X_test.shape, y_test.shape

((574, 29), (574,))

In [ ]:
df.shape

(2293, 30)

Провели масштабирование числовых признаков.

In [ ]:
# cоздали объект структуры StandardScaler() и настроили его на обучающих данных
scaler = StandardScaler()
scaler.fit(X_train)

# преобразовали обучающую и тестовую выборки функцией transform()
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Написали функцию перебора гиперпараметров по сетке со своим скорером.

In [ ]:
# функция для оценки качества модели в GridSearch
my_scorer = make_scorer(score_func=mean_absolute_error, greater_is_better=False)

def grid_search(model, param_grid, cv, features_train, target_train):
    grid_model = GridSearchCV(model, param_grid=param_grid, cv=cv, scoring=my_scorer, n_jobs=-1)
    grid_model.fit(features_train, target_train)
    best_estimator = grid_model.best_estimator_
    best_parameters = grid_model.best_params_
    best_score = grid_model.best_score_
    return best_estimator, best_parameters, best_score

***

### 3.1 Линейная регрессия

In [ ]:
# сетка гиперпараметров линейной регрессии
param_grid_reg={'fit_intercept':[True, False],
                'normalize':[True, False]}

In [ ]:
lin_reg = LinearRegression()
grid_lin_reg = grid_search(lin_reg, param_grid_reg, 5, X_train, y_train)
grid_lin_reg

(LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True),
 {'fit_intercept': True, 'normalize': True},
 -6.536510350394517)

In [ ]:
# определение mae
pred_lin_reg = grid_lin_reg[0].predict(X_test)
lin_reg_mae = mean_absolute_error(y_test, pred_lin_reg)
lin_reg_mae

6.19743440057547

***

### 3.2 Случайный лес

In [ ]:
# сетка параметров случайного леса
param_grid_forest={'n_estimators':[25, 50, 75],
                   'max_depth':[3, 4, 5],
                   'max_features': ['auto', 'sqrt']}

In [ ]:
rfr = RandomForestRegressor(random_state=12345)
grid_rfr = grid_search(rfr, param_grid_forest, 5, X_train, y_train)
grid_rfr

(RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=75,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False),
 {'max_depth': 5, 'max_features': 'auto', 'n_estimators': 75},
 -6.370481213589712)

In [ ]:
# определение mae
pred_rfr = grid_rfr[0].predict(X_test)
rfr_mae = mean_absolute_error(y_test, pred_rfr)
rfr_mae

6.469991554435425

***

### 3.3 Градиентный бустинг CatBoost 

In [ ]:
cat = CatBoostRegressor(learning_rate=0.1, depth=2, verbose=False, random_state=12345)

In [ ]:
# обучение и определение метрики mae
cat.fit(X_train, y_train)
predictions = cat.predict(X_test)
cat_mae = mean_absolute_error(y_test, predictions)
cat_mae

5.865325926441813

Свели метрики качества разных моделей в одну таблицу.

In [ ]:
score = pd.DataFrame({'lin_reg': [lin_reg_mae], 'random_forest': [rfr_mae], 'catboost': [cat_mae]}, index=['mae'])
score

,lin_reg,random_forest,catboost
mae,6.197434,6.469992,5.865326


#### Вывод   

Лучшая метрика mae у модели градиентного бустинга - 5,8. Данный параметр укладывается в диапазон допустимых значений метрики качества. Линейная регрессия неожиданно показала более лучший score, чем случайный лес.

***

## 4. Отчёт

Первоначальным условием оптимизации потребления электрической энергии при выплавке стали было создание модели, предсказывающую конечную температуру при каждом итерационном процессе. Признаком успешности модели являлось минимальное значение средней абсолютной ошибки MAE предсказанной и истинной конечной температуры в пределах, установленных заданием (<8,7).  

* В результате предобработки данных мы просуммировали затраченную энергию на нагрев стали и общее время нагрева для предсказания конечной темпереатуры в ковше в пределах иттерационного процесса. Особо критичных выбросов в данных не обнаружили. В качестве целевого признака выбрали конечную температуру нагрева, а в качестве одного из признаков - первичную измеренную температуру. Данные о подаче материалов в качестве присадок в стали имеют техническое обоснование, поэтому пропуски здесь закономерны и заполнены нулем. В итоге совместили все признаки и таргет в нескольких таблицах в одну общую для дальнейшего анализа и обучения моделей.  


* При соединении нескольких таблиц по ключу `key` в общей таблице у нас образовались пропуски. Причём пропуски соотносились с разными диапазонами ключей `key`. В итоге приняли решение удалить эти строки, а не заполнить их нулем из-за того, что строки со всеми нулями в признаках мало повлияли бы на конечную модель.   


* При анализе мы также обнаружили ряд значений первичной температуры в признаках, совпадающей с конечной в таргете. Во избежании утечки удалили эти строки. В результате очистки наша таблица лишилась ~25% строк.  


* Обучили несколько моделей, предварительно промасштабировав численные признаки. Линейная регрессия, случайный лес и градиентный бустинг показали хорошие результаты по метрике MAE. Лучшей моделью, как ни странно, оказалась CatBoost со score 5,8.


* Модель CatBoost успешно может применяться для адекватного предсказания конечной температуры в пределах заданного диапазона метрики качества.

In [ ]:
# сравнительная таблица
score

,lin_reg,random_forest,catboost
mae,6.197434,6.469992,5.865326
